In [1]:
import pandas as pd
from preprocess import preprocess_tweet
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer
from string import punctuation, digits
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_validate
from sklearn.feature_extraction.text import (
    CountVectorizer, TfidfVectorizer)
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import (
    SelectKBest, VarianceThreshold, f_classif)
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
import tensorflow as tf
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score

## Load data 

In [2]:
n_data = []
p_data = []

# read positive and negative tweet data
# use own path here
with open('/Users/louancillon/Documents/ETHZ/M2/CIL/ETH_CIL22/twitter-datasets/train_neg.txt', encoding='utf-8') as f:
    lines = f.read().splitlines()
    n_data = lines
print('read negative tweet lines ', len(n_data))

with open('/Users/louancillon/Documents/ETHZ/M2/CIL/ETH_CIL22/twitter-datasets/train_pos.txt', encoding='utf-8') as f:
    lines = f.read().splitlines()
    p_data = lines
print('read positive tweet lines ', len(p_data))

# fill all tweet data and target labels
tweet_data = p_data + n_data
tweet_labels = [1.0 for _ in p_data] + [0.0 for _ in n_data]
print(len(tweet_data))
print(len(tweet_labels))


read negative tweet lines  100000
read positive tweet lines  100000
200000
200000


In [3]:
df = pd.DataFrame({'review':tweet_data, 'sentiment':tweet_labels})
original = df['review'].map(lambda x: x.split())
df.insert(1, 'original',original)
x_train, x_test,y_train, y_test = train_test_split(df['original'], df['sentiment'], test_size=0.1, random_state = 45)


## NN baseline 

In [4]:
# Set Maximum number of words to be embedded
NUM_WORDS = 25000

# Define/Load Tokenize text function
tokenizer = Tokenizer(num_words=NUM_WORDS,lower=True)

# Fit the function on the text
tokenizer.fit_on_texts(x_train)

# Count number of unique tokens
word_index = tokenizer.word_index

# Convert train and val to sequence
sequences_train = tokenizer.texts_to_sequences(x_train)
# Limit size of train/val to 50 and pad the sequence
x_train = pad_sequences(sequences_train,maxlen=50)

In [16]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(
        input_dim=NUM_WORDS,
        output_dim=64,
        # Use masking to handle the variable sequence lengths
        mask_zero=True),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1)
])

In [17]:
# Compile the Model
model.compile(loss='binary_crossentropy', optimizer='adam', 
          metrics=['acc'])


In [18]:
# Train the Model
model.fit(x_train, y_train, epochs=5, batch_size=25, verbose = 1)

Epoch 1/5
7200/7200 [==============================] - 209s 28ms/step - loss: 0.4488 - acc: 0.8022
Epoch 2/5
7200/7200 [==============================] - 229s 32ms/step - loss: 0.3883 - acc: 0.8399
Epoch 3/5
7200/7200 [==============================] - 244s 34ms/step - loss: 0.3848 - acc: 0.8469
Epoch 4/5
7200/7200 [==============================] - 245s 34ms/step - loss: 0.3783 - acc: 0.8522
Epoch 5/5
7200/7200 [==============================] - 250s 35ms/step - loss: 0.3556 - acc: 0.8686


In [14]:
# Convert train and val to sequence
sequences_test = tokenizer.texts_to_sequences(x_test)
# Limit size of train/val to 50 and pad the sequence
X_test = pad_sequences(sequences_test,maxlen=50)


In [19]:
# evaluate
loss, acc = model.evaluate(X_test, y_test, verbose=0)
print('Test Accuracy: %f' % (acc*100))

Test Accuracy: 82.870001


In [13]:
kfold = KFold(n_splits=10)
scores = np.zeros(10)
idx = 0
VOCAB_SIZE = 25000
X = df['original']
Y = df['sentiment']

for train, test in kfold.split(X, Y):

    encoder = tf.keras.layers.TextVectorization(max_tokens=VOCAB_SIZE)
    encoder.adapt(X[train].map(lambda text, label: text))


    # Define a Deep Learning Model
    model = tf.keras.Sequential([
        encoder,
        tf.keras.layers.Embedding(
            input_dim=len(encoder.get_vocabulary()),
            output_dim=64,
            # Use masking to handle the variable sequence lengths
            mask_zero=True),
        tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dense(1)
    ])

    # Compile the Model
    model.compile(loss='binary_crossentropy', optimizer='adam', 
              metrics=['acc'])

    # Train the Model
    model.fit(X[train], y[train], epochs=50, batch_size=25, verbose = 0,
              validation_data=(X[test], y[test]))

    # evaluate the model
    scores[idx] = train_evaluate(model, X[train], Y[train], X[test], Y[test])
    print(scores[idx])
    idx += 1
    


TypeError: <lambda>() missing 1 required positional argument: 'label'

In [ ]:
def train_eval(x, y, model):
    # Create a pipeline
    baseline = Pipeline([
      ("tf-idf", TfidfVectorizer(lowercase=False, tokenizer=lambda i:i)),
       #TfidfVectorizer(lowercase=False)),
      ("clf", model)
    ])
    
    cv_results = cross_validate(baseline, x_train, y_train, cv=10)
    
    return cv_results
    

In [ ]:
cv_results = cross_validate(baseline, x_train, y_train, cv=10)

In [ ]:
cv_results